# Derive models of spacecraft with star tracker

Do all imports.

In [1]:
import sympy as sym
import numpy as np
from scipy import linalg
from scipy import signal
import ae353_spacecraft_design as design

## Create spacecraft

Create a visualizer to help with placement of reaction wheels.

In [2]:
# Create the visualizer
vis = design.create_visualizer()

# Show the visualizer in this notebook
vis.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


Specify the location of each reaction wheel in terms of its right ascension $\alpha$ and declination $\delta$:

In [3]:
wheels = [
    {'alpha': 1 * np.pi / 4, 'delta': np.pi},
    {'alpha': 3 * np.pi / 4, 'delta': -np.pi},
    {'alpha': 5 * np.pi / 4, 'delta': np.pi/2},
    {'alpha': 7 * np.pi / 4, 'delta': -np.pi/2},
]

Show wheels in the visualizer. You will be warned if any wheel obscures the star tracker (i.e., the "scope") or if any two wheels are too close together.

In [4]:
design.show_wheels(vis, wheels)

Create a model of the spacecraft in URDF format. This will **overwrite** the file `spacecraft.urdf` in the `urdf` directory.

In [5]:
design.create_spacecraft(wheels)

## Create dynamic model

Specify the physical parameters:

In [6]:
# Mass and MOI of base
mb = 6.
Jxb = 10.
Jyb = 10.
Jzb = 16.

# Mass and MOI of each wheel
mw = 1.
Jxw = 0.075
Jyw = 0.075
Jzw = 0.125
lw = 2.2

Derive the equations of motion:

In [7]:
# Define yaw, pitch, roll angles
psi, theta, phi = sym.symbols('psi, theta, phi')

# Define angular velocities
w_x, w_y, w_z = sym.symbols('w_x, w_y, w_z')

# Define torques
tau_1, tau_2, tau_3, tau_4 = sym.symbols('tau_1, tau_2, tau_3, tau_4')

# Compute resultant torques
T1 = - tau_1 * sym.Matrix(wheels[0]['xyz'])
T2 = - tau_2 * sym.Matrix(wheels[1]['xyz'])
T3 = - tau_3 * sym.Matrix(wheels[2]['xyz'])
T4 = - tau_4 * sym.Matrix(wheels[3]['xyz'])
T = sym.nsimplify(T1 + T2 + T3 + T4)

# Define MOI of spacecraft and wheels together
#
#  FIXME: For now, assume that each RW is a point mass. Later,
#         somebody should do this properly.
#
Jx = sym.nsimplify(Jxb + 4 * mw * lw**2)
Jy = sym.nsimplify(Jyb + 4 * mw * lw**2)
Jz = sym.nsimplify(Jzb + 4 * mw * lw**2)

# Define rotation matrices
Rz = sym.Matrix([[sym.cos(psi), -sym.sin(psi), 0], [sym.sin(psi), sym.cos(psi), 0], [0, 0, 1]])
Ry = sym.Matrix([[sym.cos(theta), 0, sym.sin(theta)], [0, 1, 0], [-sym.sin(theta), 0, sym.cos(theta)]])
Rx = sym.Matrix([[1, 0, 0], [0, sym.cos(phi), -sym.sin(phi)], [0, sym.sin(phi), sym.cos(phi)]])

# Define the transformation from angular velocity to angular rates
ex = sym.Matrix([[1], [0], [0]])
ey = sym.Matrix([[0], [1], [0]])
ez = sym.Matrix([[0], [0], [1]])
M = sym.simplify(sym.Matrix.hstack((Ry @ Rx).T @ ez, Rx.T @ ey, ex).inv(), full=True)

# Define euler's equations
euler = sym.Matrix([[(1 / Jx) * (T[0] + (Jy - Jz) * w_y * w_z)],
                   [(1 / Jy) * (T[1] + (Jz - Jx) * w_z * w_x)],
                   [(1 / Jz) * (T[2] + (Jx - Jy) * w_x * w_y)]])

# Define equations of motion
f = sym.simplify(sym.Matrix.vstack(M * sym.Matrix([[w_x], [w_y], [w_z]]), euler), full=True)

The equations of motion have this form:

$$\begin{bmatrix} \dot{\psi} \\ \dot{\theta} \\ \dot{\phi} \\ \dot{w_x} \\ \dot{w_y} \\ \dot{w_z} \end{bmatrix} = f\left(\psi, \theta, \phi, w_x, w_y, w_z, \tau_1, \tau_2, \tau_3, \tau_4\right)$$

Here is the function $f$:

In [8]:
f

Matrix([
[                                                                                                                                                                    (w_y*sin(phi) + w_z*cos(phi))/cos(theta)],
[                                                                                                                                                                                 w_y*cos(phi) - w_z*sin(phi)],
[                                                                                                                                                     w_x + w_y*sin(phi)*tan(theta) + w_z*cos(phi)*tan(theta)],
[1944543648263*tau_1/36700000000000 - 1944543648263*tau_2/36700000000000 + 952551661859043*tau_3/293600000000000000000000000000000 - 476275830929521*tau_4/146800000000000000000000000000000 - 75*w_y*w_z/367],
[1944543648263*tau_1/36700000000000 + 1944543648263*tau_2/36700000000000 + 476275830929521*tau_3/146800000000000000000000000000000 + 952551661859043*tau_4/2936

In [9]:
# equilibrium values of each variable, calculated by hand
psi_e = 0 # angle
theta_e = 0 # angle
phi_e = 0 # angle
w_xe = 0
w_ye = 0 
w_ze = 0
tau_1e = 0
tau_2e = 0
tau_3e = 0
tau_4e = 0


A_num = sym.lambdify((psi,theta, phi,w_x,w_y,w_z,tau_1, tau_2, tau_3, tau_4), f.jacobian([psi,theta, phi,w_x,w_y,w_z]))
B_num = sym.lambdify((psi,theta, phi,w_x,w_y,w_z,tau_1, tau_2, tau_3, tau_4), f.jacobian([tau_1, tau_2, tau_3, tau_4]))

A = A_num(psi_e,theta_e,phi_e,w_xe,w_ye,w_ze,tau_1e, tau_2e, tau_3e, tau_4e)
B = B_num(psi_e,theta_e,phi_e,w_xe,w_ye,w_ze,tau_1e, tau_2e, tau_3e, tau_4e)

A = A.astype(float)
B = B.astype(float)

In [10]:
A,B

(array([[ 0.,  0.,  0.,  0.,  0.,  1.],
        [ 0.,  0.,  0.,  0.,  1., -0.],
        [ 0.,  0.,  0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  0., -0., -0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.]]),
 array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 5.29848406e-02, -5.29848406e-02,  3.24438577e-18,
         -3.24438577e-18],
        [ 5.29848406e-02,  5.29848406e-02,  3.24438577e-18,
          3.24438577e-18],
        [-7.61940882e-18,  7.61940882e-18, -6.22171946e-02,
          6.22171946e-02]]))

In [11]:
W = np.block([B,A@B,A@A@B, A@A@A@B, A@A@A@A@B, A@A@A@A@A@B])

In [12]:
np.linalg.matrix_rank(W)

6

In [13]:
Am = sym.Matrix(A)
Bm = sym.Matrix(B)
Am, Bm

(Matrix([
 [0, 0, 0,   0,   0, 1.0],
 [0, 0, 0,   0, 1.0,   0],
 [0, 0, 0, 1.0,   0,   0],
 [0, 0, 0,   0,   0,   0],
 [0, 0, 0,   0,   0,   0],
 [0, 0, 0,   0,   0,   0]]),
 Matrix([
 [                    0,                    0,                    0,                     0],
 [                    0,                    0,                    0,                     0],
 [                    0,                    0,                    0,                     0],
 [   0.0529848405521253,  -0.0529848405521253, 3.24438576927467e-18, -3.24438576927467e-18],
 [   0.0529848405521253,   0.0529848405521253, 3.24438576927467e-18,  3.24438576927467e-18],
 [-7.61940881822449e-18, 7.61940881822449e-18,  -0.0622171945701357,    0.0622171945701357]]))

In [14]:
def lqr(A, B, Q, R):
    P = linalg.solve_continuous_are(A, B, Q, R)
    K = linalg.inv(R) @ B.T @ P
    return K

In [15]:
q1 = 1
q2 = 1
q3 = 1
q4 = 1
q5 = 1
q6 = 1

Q = np.diag([q1,q2,q3,q4,q5,q6])
R = np.eye(4)

In [16]:
K = lqr(A,B,Q,R)

In [17]:
K

array([[-7.16435136e-16,  7.07106781e-01,  7.07106781e-01,
         3.72094807e+00,  3.72094807e+00, -8.72057919e-16],
       [-1.35350397e-16,  7.07106781e-01, -7.07106781e-01,
        -3.72094807e+00,  3.72094807e+00, -9.00120736e-16],
       [-7.07106781e-01,  1.22393720e-16, -7.68463772e-17,
        -2.83976895e-16,  1.26832849e-15, -3.44458045e+00],
       [ 7.07106781e-01, -3.57981146e-17,  7.68463772e-17,
         2.83976895e-16, -8.12643780e-16,  3.44458045e+00]])

## Create sensor model

Symbolic variables for right ascension $\alpha$ and declination $\delta$ of each star:

In [18]:
alpha, delta = sym.symbols('alpha, delta')

Specify the physical parameters:

In [19]:
# Scope radius
r = 0.8 / 2.1

Derive the sensor model:

In [35]:
# Position of star in space frame
p_star_in_space = sym.Matrix([[sym.cos(alpha) * sym.cos(delta)],
                              [sym.sin(alpha) * sym.cos(delta)],
                              [sym.sin(delta)]])

# Orientation of body frame in space frame
R_body_in_space = Rz * Ry * Rx

# Position of star in body frame (assuming origin of body and space frames are the same)
p_star_in_body = R_body_in_space.T * p_star_in_space

# Position of star in image frame
p_star_in_image = (1 / sym.nsimplify(r)) * sym.Matrix([[p_star_in_body[1] / p_star_in_body[0]],
                                                       [p_star_in_body[2] / p_star_in_body[0]]])

# Sensor model for each star
g = sym.simplify(p_star_in_image, full=True)

The sensor model has this form for each star:

$$\zeta = g(\psi, \theta, \phi, \alpha, \delta)$$

Here is the function $g$:

In [21]:
g

Matrix([
[21*(sin(delta)*sin(phi)*cos(theta) + sin(phi)*sin(theta)*cos(delta)*cos(alpha - psi) + sin(alpha - psi)*cos(delta)*cos(phi))/(8*(-sin(delta)*sin(theta) + cos(delta)*cos(theta)*cos(alpha - psi)))],
[21*(sin(delta)*cos(phi)*cos(theta) - sin(phi)*sin(alpha - psi)*cos(delta) + sin(theta)*cos(delta)*cos(phi)*cos(alpha - psi))/(8*(-sin(delta)*sin(theta) + cos(delta)*cos(theta)*cos(alpha - psi)))]])

In [34]:
alpha0 = -0.10 
delta0 = -0.15
alpha1 =  0.00
delta1 = -0.15
alpha2 =  0.10 
delta2 = -0.15
alpha3 =  0.00
delta3 =  0.00
alpha4 = -0.10
delta4 =  0.15
alpha5 =  0.00
delta5 =  0.15
alpha6 =  0.10
delta6 =  0.15

In [39]:
g0 = g.subs(alpha, alpha0).subs(delta,delta0)
g1 = g.subs(alpha, alpha1).subs(delta,delta1)
g2 = g.subs(alpha, alpha2).subs(delta,delta2)
g3 = g.subs(alpha, alpha3).subs(delta,delta3)
g4 = g.subs(alpha, alpha4).subs(delta,delta4)
g5 = g.subs(alpha, alpha5).subs(delta,delta5)
g6 = g.subs(alpha, alpha6).subs(delta,delta6)

In [41]:
gprime = sym.Matrix([g0,g1,g2,g3,g4,g5,g6])

In [42]:
gprime

Matrix([
[ 21*(0.988771077936042*sin(phi)*sin(theta)*cos(psi + 0.1) - 0.149438132473599*sin(phi)*cos(theta) - 0.988771077936042*sin(psi + 0.1)*cos(phi))/(8*(0.149438132473599*sin(theta) + 0.988771077936042*cos(theta)*cos(psi + 0.1)))],
[ 21*(0.988771077936042*sin(phi)*sin(psi + 0.1) + 0.988771077936042*sin(theta)*cos(phi)*cos(psi + 0.1) - 0.149438132473599*cos(phi)*cos(theta))/(8*(0.149438132473599*sin(theta) + 0.988771077936042*cos(theta)*cos(psi + 0.1)))],
[                   21*(0.988771077936042*sin(phi)*sin(theta)*cos(psi) - 0.149438132473599*sin(phi)*cos(theta) - 0.988771077936042*sin(psi)*cos(phi))/(8*(0.149438132473599*sin(theta) + 0.988771077936042*cos(psi)*cos(theta)))],
[                   21*(0.988771077936042*sin(phi)*sin(psi) + 0.988771077936042*sin(theta)*cos(phi)*cos(psi) - 0.149438132473599*cos(phi)*cos(theta))/(8*(0.149438132473599*sin(theta) + 0.988771077936042*cos(psi)*cos(theta)))],
[ 21*(0.988771077936042*sin(phi)*sin(theta)*cos(psi - 0.1) - 0.149438132473599*sin(

In [54]:
C_num = sym.lambdify((psi,theta, phi,w_x,w_y,w_z,tau_1, tau_2, tau_3, tau_4),gprime.jacobian([psi,theta, phi,w_x,w_y,w_z]))
D_num = sym.lambdify((psi,theta, phi,w_x,w_y,w_z,tau_1, tau_2, tau_3, tau_4),gprime.jacobian([tau_1, tau_2, tau_3, tau_4]))

In [55]:
C_num

<function _lambdifygenerated(psi, theta, phi, w_x, w_y, w_z, tau_1, tau_2, tau_3, tau_4)>

In [56]:
C = C_num(psi_e,theta_e,phi_e,w_xe,w_ye,w_ze,tau_1e, tau_2e, tau_3e, tau_4e)
D = D_num(psi_e,theta_e,phi_e,w_xe,w_ye,w_ze,tau_1e, tau_2e, tau_3e, tau_4e)

C = C.astype(float)
D = D.astype(float)

In [57]:
C

array([[-2.651426  ,  0.04000563, -0.3987219 ,  0.        ,  0.        ,
         0.        ],
       [-0.04000563,  2.68556349,  0.26337851,  0.        ,  0.        ,
         0.        ],
       [-2.625     ,  0.        , -0.39672995,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  2.68495987,  0.        ,  0.        ,  0.        ,
         0.        ],
       [-2.651426  , -0.04000563, -0.3987219 ,  0.        ,  0.        ,
         0.        ],
       [ 0.04000563,  2.68556349, -0.26337851,  0.        ,  0.        ,
         0.        ],
       [-2.625     ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  2.625     ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [-2.651426  , -0.04000563,  0.3987219 ,  0.        ,  0.        ,
         0.        ],
       [ 0.04000563,  2.68556349,  0.26337851,  0.        ,  0.        ,
         0.        ],
       [-2.625     ,  0.        ,  0.39672995,  0.